# Import Libraries

In [11]:
import numpy as np
import pandas as pd
import psycopg2
import os
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import plotly.express as px

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.compose import make_reduction
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, r2_score)
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.model_selection import ForecastingGridSearchCV, ForecastingRandomizedSearchCV, SlidingWindowSplitter, ExpandingWindowSplitter, SingleWindowSplitter
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanSquaredError
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor
from sktime.forecasting.fbprophet import Prophet

# Set Project Path

In [12]:
# getting the name of the directory
# where the this file is present.
current = os.path.dirname(os.path.abspath("__file__"))

# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(current)

# Getting the parent directory name
gr_parent = os.path.dirname(parent)

# adding the parent directory to
# the sys.path.
sys.path.append(gr_parent)

sys.path.insert(0, "..//skk_analytics")

In [13]:
from connection import *
from utils import *

In [14]:
file_config = gr_parent + "\\database.ini"
print(file_config)

sql_file = os.path.join(parent, 'sql\\lng_prod_tangguh_data_query.sql')
print(sql_file)

/home/spcuser/Documents/code/python/skk/analytics/develop/skk_analytics\database.ini
/home/spcuser/Documents/code/python/skk/analytics/develop/skk_analytics/gas_prod/sql\lng_prod_tangguh_data_query.sql


# Get Data from Database

In [15]:
conn = create_db_connection(filename=file_config, section='postgresql_ml_lng_skk')
if conn == None:
    exit()

#Load Data from Database
query_1 = open(sql_file, mode="rt").read()
data = get_sql_data(query_1, conn)

data['date'] = pd.DatetimeIndex(data['date'], freq='D')
data = data.reset_index()

KeyError: 'postgresql_ml_lng_skk'

In [ ]:
ds = 'date'
y = 'lng_production' 

df = data[[ds,y]]
df = df.set_index(ds)
df.index = pd.DatetimeIndex(df.index, freq='D')

# Time Series Split

In [ ]:
# Test size
test_size = 0.2
# Split data (original data)
y_train, y_test = temporal_train_test_split(df, test_size=test_size)

# Horizon
fh = ForecastingHorizon(y_test.index, is_relative=False)
fh_int = np.arange(1, len(fh))

# Create Additional Regressor (Exogenous)

In [ ]:
## Create Exogenous Variable
df['month'] = [i.month for i in df.index]
df['planned_shutdown'] = data['planned_shutdown'].values
df['day'] = [i.day for i in df.index]

 # Split into train and test
X_train, X_test = temporal_train_test_split(df.iloc[:,1:], test_size=test_size)
exogenous_features = ["month", "day", "planned_shutdown"]

# Build Model

In [ ]:
# Model scoring for Cross Validation
mape = MeanAbsolutePercentageError(symmetric=False)
mse = MeanSquaredError()

## Random Forest

### Define Model Parameters

In [ ]:
rf_n_estimators = 200
rf_lags = 27 #1, 6, 27
rf_random_state = 0
rf_criterion = "squared_error"
rf_strategy = "recursive"
#n_estimators_param_grid = {"n_estimators": [100, 150, 200, 300]}
forecaster_param_grid = {"window_length": [1, 7, 14, 21, 30], 
                         "estimator__n_estimators": [200, 300]}

# Create regressor object
#rf_regressor = RandomForestRegressor(n_estimators = rf_n_estimators, random_state = rf_random_state, criterion = rf_criterion)
rf_regressor = RandomForestRegressor(random_state = rf_random_state, criterion = rf_criterion, n_jobs=-1)
#rf_forecaster = make_reduction(rf_regressor, window_length = rf_lags, strategy = rf_strategy)
rf_forecaster = make_reduction(rf_regressor, strategy = rf_strategy)

# Define Cross Validation object
#cv = ExpandingWindowSplitter(fh=int(len(fh)), initial_window=365*2, step_length=30)
#cv = SlidingWindowSplitter(window_length=365*2, step_length=1, fh=fh_int)
cv = SingleWindowSplitter(fh=fh_int)
gscv = ForecastingGridSearchCV(rf_forecaster, cv=cv, param_grid=forecaster_param_grid, n_jobs=-1, scoring=mse)


In [ ]:
# Check splitter data split
list(cv.split_series(y_train))

### Perform Cross Validation

In [ ]:
X_train = X_train.asfreq('D')

# Perform Cross Validation Model
print("Creating Random Forest Model ...")
#rf_forecaster.fit(y_train) #, X_train
gscv.fit(y_train, X_train) #, X_train

Creating Random Forest Model ...


ForecastingGridSearchCV(cv=SingleWindowSplitter(fh=array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100,...
       456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468,
       469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481,
       482, 483, 484, 485, 486, 487, 488, 489])),
                        forecaster=RecursiveTabularRegressionForecaster(estimator=RandomForestRegressor(n_jobs=-1,
                                                                                                        random_state=0)),
                        n_jobs=-1,
                        param_grid={'estimator__n_estimators': [200, 300],
                                    'window_length': [1, 7, 14, 21, 30]},
                        scoring=MeanSquaredError())

In [ ]:
# Show top 10 best models based on scoring function
gscv.cv_results_.sort_values(by='rank_test_MeanSquaredError')

,mean_test_MeanSquaredError,mean_fit_time,mean_pred_time,params,rank_test_MeanSquaredError
3,8.761794e+07,3.389005,19.683351,"{'estimator__n_estimators': 200, 'window_lengt...",1.0
6,1.026314e+08,2.822040,24.709442,"{'estimator__n_estimators': 300, 'window_lengt...",2.0
5,1.604116e+08,3.837962,23.684362,"{'estimator__n_estimators': 300, 'window_lengt...",3.0
1,1.920462e+08,0.705899,22.208366,"{'estimator__n_estimators': 200, 'window_lengt...",4.0
0,1.982373e+08,0.568499,22.246202,"{'estimator__n_estimators': 200, 'window_lengt...",5.0
8,2.191878e+08,5.066274,22.371141,"{'estimator__n_estimators': 300, 'window_lengt...",6.0
9,2.274003e+08,5.598472,21.782025,"{'estimator__n_estimators': 300, 'window_lengt...",7.0
4,2.371658e+08,4.898882,17.971345,"{'estimator__n_estimators': 200, 'window_lengt...",8.0
7,2.777457e+08,5.302227,22.160754,"{'estimator__n_estimators': 300, 'window_lengt...",9.0
2,2.805430e+08,0.922029,21.925065,"{'estimator__n_estimators': 200, 'window_lengt...",10.0


In [ ]:
# Show best model parameters
gscv.best_params_

{'estimator__n_estimators': 200, 'window_length': 21}

### Perform Prediction Based on Best Model

In [ ]:
print("Random Forest Model Prediction ...")
#rf_forecast = rf_forecaster.predict(fh) #, X=X_test
rf_forecast = gscv.best_forecaster_.predict(fh, X=X_test)#, X=X_test

Random Forest Model Prediction ...


### Model Performance

In [ ]:
# Create MAPE
y_pred_rf = pd.DataFrame(rf_forecast).applymap('{:.2f}'.format)
rf_mape = mean_absolute_percentage_error(y_test['lng_production'], y_pred_rf)
ranfor_mape_str = str('MAPE: %.4f' % rf_mape)
print("Random Forest Model "+ranfor_mape_str)

Random Forest Model MAPE: 0.2053


In [ ]:
#Get Parameters
#rf_param = str(rf_forecaster.get_params())
rf_param = str(gscv.get_params())
print("Random Forest Model Parameters "+rf_param)

## XGBoost

### Define Model Parameters

In [ ]:
xgb_objective = 'reg:squarederror'
xgb_lags = 6 #1, 6, 27
xgb_strategy = "recursive"
xgb_forecaster_param_grid = {"window_length": [1, 6, 7, 14, 21, 27, 32]
                            ,"estimator__n_estimators": [100, 200, 300]
                            #,"estimator__max_depth": [3,6,10],
                            #,"estimator__learning_rate": [0.01, 0.05, 0.1, 0.3],
                            #,"estimator__colsample_bytree": [0.3, 0.5, 0.7]
                            }

xgb_regressor = XGBRegressor(objective=xgb_objective, n_jobs=-1, seed = 42)
xgb_forecaster = make_reduction(xgb_regressor, strategy=xgb_strategy)

cv_xgb = SingleWindowSplitter(fh=fh_int)
#gscv_xgb = ForecastingRandomizedSearchCV(xgb_forecaster, cv=cv_xgb, param_grid=xgb_forecaster_param_grid, n_jobs=-1, scoring=mape)
gscv_xgb = ForecastingGridSearchCV(xgb_forecaster, cv=cv_xgb, param_grid=xgb_forecaster_param_grid, n_jobs=-1, scoring=mse)


In [ ]:
# Check splitter data split
#list(cv_xgb.split_series(y_train))

### Perform Cross Validation

In [ ]:
X_train = X_train.asfreq('D')

# Perform Cross Validation Model
print("Creating XGBoost Model ...")
#rf_forecaster.fit(y_train) #, X_train
gscv_xgb.fit(y_train, X_train) #, X_train

Creating XGBoost Model ...


ForecastingGridSearchCV(cv=SingleWindowSplitter(fh=array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100,...
                                                                                               max_cat_threshold=None,
                                                                                               max_cat_to_onehot=None,
                                                                                               max_delta_step=None,
                                                                                               max_depth=None,
                                                                                               max_leaves=None,
                                                                                               min_child_weight=None,
                                                                                               missing=nan,
                                                                                               monotone_constraints=None,
                                                                                               n_estimators=100,
                                                                                               n_jobs=-1,
                                                                                               num_parallel_tree=None,
                                                                                               predictor=None,
                                                                                               random_state=None, ...)),
                        n_jobs=-1,
                        param_grid={'estimator__n_estimators': [100, 200, 300],
                                    'window_length': [1, 6, 7, 14, 21, 27, 32]},
                        scoring=MeanSquaredError())

In [ ]:
# Show top 10 best models based on scoring function
gscv_xgb.cv_results_.sort_values(by='rank_test_MeanSquaredError', ascending=True)

,mean_test_MeanSquaredError,mean_fit_time,mean_pred_time,params,rank_test_MeanSquaredError
0,8.195243e+07,0.116710,0.852540,"{'estimator__n_estimators': 100, 'window_lengt...",1.0
15,8.876519e+07,1.140005,0.472122,"{'estimator__n_estimators': 300, 'window_lengt...",2.0
8,9.002155e+07,0.979624,0.823793,"{'estimator__n_estimators': 200, 'window_lengt...",3.0
11,9.020255e+07,2.678577,0.587015,"{'estimator__n_estimators': 200, 'window_lengt...",4.0
1,9.283098e+07,0.447449,0.713828,"{'estimator__n_estimators': 100, 'window_lengt...",5.0
3,9.638852e+07,1.011528,0.877997,"{'estimator__n_estimators': 100, 'window_lengt...",6.0
18,9.674295e+07,2.897170,0.371088,"{'estimator__n_estimators': 300, 'window_lengt...",7.0
7,9.846953e+07,0.276975,0.797016,"{'estimator__n_estimators': 200, 'window_lengt...",8.0
20,1.051913e+08,4.736620,0.372429,"{'estimator__n_estimators': 300, 'window_lengt...",9.0
14,1.072697e+08,0.340952,0.570779,"{'estimator__n_estimators': 300, 'window_lengt...",10.0


In [ ]:
# Show best model parameters
gscv_xgb.best_params_

{'estimator__n_estimators': 100, 'window_length': 1}

### Perform Prediction Based on Best Model

In [ ]:
print("XGBoost Model Prediction ...")
#rf_forecast = rf_forecaster.predict(fh) #, X=X_test
xgb_forecast = gscv_xgb.best_forecaster_.predict(fh, X=X_test)#, X=X_test

XGBoost Model Prediction ...


### Model Performance

In [ ]:
# Create MAPE
y_pred_xgb = pd.DataFrame(xgb_forecast).applymap('{:.2f}'.format)
#y_pred_xgb
xgb_mape = mean_absolute_percentage_error(y_test['lng_production'], y_pred_xgb)
xgb_mape_str = str('MAPE: %.4f' % xgb_mape)
print("XGBoost Model "+ xgb_mape_str)

XGBoost Model MAPE: 0.1328


In [ ]:
#Get Parameters
#rf_param = str(rf_forecaster.get_params())
xgb_param = str(gscv_xgb.get_params())
print("Random Forest Model Parameters "+xgb_param)

## Prophet

### Define Model Parameters

In [ ]:
seasonality_mode = 'additive'
n_changepoints = 27 #1, 6, 27
seasonality_prior_scale = 0.05
changepoint_prior_scale = 0.1
holidays_prior_scale = 8
daily_seasonality = 8
weekly_seasonality = 1
yearly_seasonality = 10

prophet_param_grid = {'n_changepoints':[1,5]
                      ,'seasonality_prior_scale':[0.05, 0.1]
                      ,'changepoint_prior_scale':[0.1, 0.5]
                      ,'daily_seasonality':[8,10]
                      ,'weekly_seasonality':[8,10]
                      ,'yearly_seasonality':[8,10]
                     }


# create regressor object
prophet_forecaster = Prophet()

cv_prophet = SingleWindowSplitter(fh=fh_int)
#gscv_xgb = ForecastingRandomizedSearchCV(xgb_forecaster, cv=cv_xgb, param_grid=xgb_forecaster_param_grid, n_jobs=-1, scoring=mape)
gscv_prophet = ForecastingGridSearchCV(prophet_forecaster, cv=cv_prophet, param_grid=prophet_param_grid, n_jobs=-1, scoring=mse)


In [ ]:
gscv_prophet.get_params().keys()

### Perform Cross Validation

In [254]:
#X_train = X_train.asfreq('D')

# Perform Cross Validation Model
print("Creating Prophet Model ...")
gscv_prophet.fit(y_train, X_train) #, X_train

Creating Prophet Model ...


17:57:43 - cmdstanpy - INFO - Chain [1] start processing
17:57:43 - cmdstanpy - INFO - Chain [1] done processing
17:57:43 - cmdstanpy - INFO - Chain [1] start processing
17:57:43 - cmdstanpy - INFO - Chain [1] done processing


ForecastingGridSearchCV(cv=SingleWindowSplitter(fh=array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100,...
       456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468,
       469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481,
       482, 483, 484, 485, 486, 487, 488, 489])),
                        forecaster=Prophet(), n_jobs=-1,
                        param_grid={'changepoint_prior_scale': [0.1, 0.5],
                                    'daily_seasonality': [8, 10],
                                    'n_changepoints': [1, 5],
                                    'seasonality_prior_scale': [0.05, 0.1],
                                    'weekly_seasonality': [8, 10],
                                    'yearly_seasonality': [8, 10]},
                        scoring=MeanSquaredError())

In [255]:
# Show top 10 best models based on scoring function
gscv_prophet.cv_results_.sort_values(by='rank_test_MeanSquaredError', ascending=True)

,mean_test_MeanSquaredError,mean_fit_time,mean_pred_time,params,rank_test_MeanSquaredError
11,6.142510e+07,1.611367,0.942106,"{'changepoint_prior_scale': 0.1, 'daily_season...",1.0
30,6.196114e+07,2.344171,1.252395,"{'changepoint_prior_scale': 0.1, 'daily_season...",2.0
25,6.207020e+07,1.729278,1.416128,"{'changepoint_prior_scale': 0.1, 'daily_season...",3.0
24,6.269874e+07,1.978467,1.342812,"{'changepoint_prior_scale': 0.1, 'daily_season...",4.0
14,6.315024e+07,12.897478,0.840115,"{'changepoint_prior_scale': 0.1, 'daily_season...",5.0
...,...,...,...,...,...
47,9.117120e+07,1.272289,0.692466,"{'changepoint_prior_scale': 0.5, 'daily_season...",60.0
61,9.118362e+07,0.981495,0.389753,"{'changepoint_prior_scale': 0.5, 'daily_season...",61.0
44,9.120539e+07,1.276058,0.810822,"{'changepoint_prior_scale': 0.5, 'daily_season...",62.0
40,9.163533e+07,1.741117,1.125873,"{'changepoint_prior_scale': 0.5, 'daily_season...",63.0


In [256]:
# Show best model parameters
gscv_prophet.best_params_

{'changepoint_prior_scale': 0.1,
 'daily_seasonality': 8,
 'n_changepoints': 5,
 'seasonality_prior_scale': 0.05,
 'weekly_seasonality': 10,
 'yearly_seasonality': 10}

### Perform Prediction Based on Best Model

In [257]:
print("Prophet Model Prediction ...")
#rf_forecast = rf_forecaster.predict(fh) #, X=X_test
prophet_forecast = gscv_prophet.best_forecaster_.predict(fh, X=X_test)#, X=X_test

Prophet Model Prediction ...


### Model Performance

In [258]:
# Create MAPE
y_pred_prophet = pd.DataFrame(prophet_forecast).applymap('{:.2f}'.format)
#y_pred_xgb
prophet_mape = mean_absolute_percentage_error(y_test['lng_production'], y_pred_prophet)
prophet_mape_str = str('MAPE: %.4f' % prophet_mape)
print("Prophet Model "+ prophet_mape_str)

Prophet Model MAPE: 0.1381


In [ ]:
#Get Parameters
#rf_param = str(rf_forecaster.get_params())
prophet_param = str(gscv_prophet.get_params())
print("Random Forest Model Parameters "+prophet_param)